# Optimization Modling HW1
* Team Members: Judy Yanbing Zhu (yz5206), Althea Jingyun Liu (jl14308)

# Q1-2 with 100 examples

In [3]:
from pyomo.environ import *
import pandas as pd

demand = [900, 900, 1050] # predicted demand
color = [3.0,1.0,2.9,4.9,5.0,5.3,3.0,7.9,3.0,5.0,9.0]
acid = [0.60,1.42,0.91,1.02,0.70,0.70,1.38,1.12,1.31,1.30,0.50] # in percentage
brix = [10.6,6.6,12.0,11.0,11.8,10.1,9.0,15.1,7.8,13.1,14.0]
astringency = [2.9,7.0,3.0,2.9,0.9,1.0,7.0,3.9,8.0,3.1,3.0]

shipping_cost = [100,150,0,60,75,50,120,110,90,130,115]
quantity = [1008,600,1800,252,126,315,882,252,450,315,270]
price = [550,341,825,660,660,688,484,660,330,506,556]

model = ConcreteModel()

model.x = Var(range(11),range(3),within= NonNegativeReals)

# Define Objective Function
model.obj = Objective(
    expr = sum((price[i] + shipping_cost[i])*model.x[i,j] for i in range(11) for j in range(3)), 
    sense = minimize)

# capacity
def capacity_rule(model,i):
    return sum(model.x[i,j] for j in range(3)) <= quantity[i]

model.capacity_con = Constraint(range(11), rule=capacity_rule)

# demand
def demand_rule(model,j):
    return sum(model.x[i,j] for i in range(11)) == demand[j]

model.demand_con = Constraint(range(3), rule=demand_rule)

# color1
def color1_rule(model,j):
    return (sum(color[i]*model.x[i,j]/demand[j] for i in range(11)) >= 4.5)
    
model.color1_con = Constraint(range(3), rule=color1_rule)

# color2
def color2_rule(model,j):
    return (sum(color[i]*model.x[i,j]/demand[j] for i in range(11)) <= 5.6)

model.color2_con = Constraint(range(3), rule=color2_rule)

# acid1
def acid1_rule(model,j):
    return (sum(acid[i]*model.x[i,j]/demand[j] for i in range(11)) >= 0.75)

model.acid1_con = Constraint(range(3), rule=acid1_rule)

# acid2
def acid2_rule(model,j):
    return (sum(acid[i]*model.x[i,j]/demand[j] for i in range(11)) <= 1)

model.acid2_con = Constraint(range(3), rule=acid2_rule)

# brix1
def brix1_rule(model,j):
    return (sum(brix[i]*model.x[i,j]/demand[j] for i in range(11)) >= 11.5)

model.brix1_con = Constraint(range(3), rule=brix1_rule)

# brix2
def brix2_rule(model,j):
    return (sum(brix[i]*model.x[i,j]/demand[j] for i in range(11)) <= 12.5)

model.brix2_con = Constraint(range(3), rule=brix2_rule)

# astringency
def astringency_rule(model,j):
    return (sum(astringency[i]*model.x[i,j]/demand[j] for i in range(11)) <= 4)

model.astringency_con = Constraint(range(3), rule=astringency_rule)

# florida
def florida_rule(model,j):
    return (model.x[2,j] >= 0.35*demand[j])

model.florida_con = Constraint(range(3), rule=florida_rule)

# x greater than 0
def x_rule(model,i,j):
    return model.x[i,j] >= 0

model.x_con = Constraint(range(11),range(3), rule=x_rule)

# solve model
opt = SolverFactory('glpk').solve(model)

res = pd.DataFrame()
for i in range(11):
    for j in range(3):
        res.at[i,j] = value(model.x[i,j])
        

res = res.set_axis(["Brazil","India","Florida","California","Arizona","Texas",
                    "China","Spain","Mexico","Egypt","Italy"], axis=0)
res = res.set_axis(["Jan", "Feb", "Mar"],axis = 1)

print('Total Cost:', value(model.obj))

print('===========================================')
print(res)
print('-------------------------------------------')

Total Cost: 1981597.2321428563
                   Jan         Feb         Mar
Brazil        0.000000    0.000000    0.000000
India         0.000000   25.982143    0.000000
Florida     315.000000  315.000000  367.500000
California    0.000000  114.428571    0.000000
Arizona       0.000000    0.000000    0.000000
Texas        55.920920   31.573896  227.505184
China         0.000000  110.089286    0.000000
Spain         6.059435   69.490292  176.450273
Mexico      197.631993   72.746699  179.621307
Egypt       182.283815   40.301767   92.414419
Italy       143.103837  120.387346    6.508817
-------------------------------------------


# Q1-3

In [19]:
0.05 * 1981597.2321428563

99079.86160714283

In [30]:
0.95 *1981597.2321428563

1882517.3705357134

In [24]:
from pyomo.environ import *
import pandas as pd
from statistics import mean

demand = [900, 900, 1050] # predicted demand
color = [3.0,1.0,2.9,4.9,5.0,5.3,3.0,7.9,3.0,5.0,9.0]
acid = [0.60,1.42,0.91,1.02,0.70,0.70,1.38,1.12,1.31,1.30,0.50] # in percentage
brix = [10.6,6.6,12.0,11.0,11.8,10.1,9.0,15.1,7.8,13.1,14.0]
astringency = [2.9,7.0,3.0,2.9,0.9,1.0,7.0,3.9,8.0,3.1,3.0]

shipping_cost = [100,150,0,60,75,50,120,110,90,130,115]
quantity = [1008,600,1800,252,126,315,882,252,450,315,270]
price = [550,341,825,660,660,688,484,660,330,506,556]

model = ConcreteModel()
model.dual = Suffix(direction = Suffix.IMPORT)

model.x = Var(range(11),range(3),within= NonNegativeReals)

# Define Objective Function
model.obj = Objective(
    expr = sum((price[i] + shipping_cost[i])*model.x[i,j] for i in range(11) for j in range(3)), 
    sense = minimize)

# capacity
def capacity_rule(model,i):
    return sum(model.x[i,j] for j in range(3)) <= quantity[i]
model.capacity_con = Constraint(range(11), rule=capacity_rule)

# demand
def demand_rule(model,j):
    return sum(model.x[i,j] for i in range(11)) == demand[j]
model.demand_con = Constraint(range(3), rule=demand_rule)

# color1
def color1_rule(model,j):
    return (sum(color[i]*model.x[i,j]/demand[j] for i in range(11)) >= 4.5)
model.color1_con = Constraint(range(3), rule=color1_rule)

# color2
def color2_rule(model,j):
    return (sum(color[i]*model.x[i,j]/demand[j] for i in range(11)) <= 5.6)
model.color2_con = Constraint(range(3), rule=color2_rule)

# acid1
def acid1_rule(model,j):
    return (sum(acid[i]*model.x[i,j]/demand[j] for i in range(11)) >= 0.75)
model.acid1_con = Constraint(range(3), rule=acid1_rule)

# acid2
def acid2_rule(model,j):
    return (sum(acid[i]*model.x[i,j]/demand[j] for i in range(11)) <= 1)
model.acid2_con = Constraint(range(3), rule=acid2_rule)

# brix1
def brix1_rule(model,j):
    return (sum(brix[i]*model.x[i,j]/demand[j] for i in range(11)) >= 11.5)
model.brix1_con = Constraint(range(3), rule=brix1_rule)

# brix2
def brix2_rule(model,j):
    return (sum(brix[i]*model.x[i,j]/demand[j] for i in range(11)) <= 12.5)
model.brix2_con = Constraint(range(3), rule=brix2_rule)

# astringency
def astringency_rule(model,j):
    return (sum(astringency[i]*model.x[i,j]/demand[j] for i in range(11)) <= 4)
model.astringency_con = Constraint(range(3), rule=astringency_rule)

# florida
def florida_rule(model,j):
    return (model.x[2,j] >= 0.35*demand[j])
model.florida_con = Constraint(range(3), rule=florida_rule)

# x greater than 0
def x_rule(model,i,j):
    return model.x[i,j] >= 0
model.x_con = Constraint(range(11),range(3), rule=x_rule)

# solve model
opt = SolverFactory('glpk').solve(model)

print('Dual Solutions')
print("dual solution of astringency", mean(model.dual[model.astringency_con[i]] for i in range(3)))
print("dual solution of brix upper limit", mean(model.dual[model.brix2_con[i]] for i in range(3)))
print("dual solution of brix lower limit", mean(model.dual[model.brix1_con[i]] for i in range(3)))
print("dual solution of acid upper limit", mean(model.dual[model.acid2_con[i]] for i in range(3)))
print("dual solution of acid lower limit", mean(model.dual[model.acid1_con[i]] for i in range(3)))
print("dual solution of color upper limit", mean(model.dual[model.color2_con[i]] for i in range(3)))
print("dual solution of color lower limit", mean(model.dual[model.color1_con[i]] for i in range(3)))

Dual Solutions
dual solution of astringency -5.033793629711757e-10
dual solution of brix upper limit 0.0
dual solution of brix lower limit 5.044110753722114e-10
dual solution of acid upper limit -25520.1863353973
dual solution of acid lower limit 0.0
dual solution of color upper limit 0.0
dual solution of color lower limit 53164.59627329097


In [37]:
from pyomo.environ import *
import pandas as pd

demand = [900, 900, 1050] # predicted demand
color = [3.0,1.0,2.9,4.9,5.0,5.3,3.0,7.9,3.0,5.0,9.0]
acid = [0.60,1.42,0.91,1.02,0.70,0.70,1.38,1.12,1.31,1.30,0.50] # in percentage
brix = [10.6,6.6,12.0,11.0,11.8,10.1,9.0,15.1,7.8,13.1,14.0]
astringency = [2.9,7.0,3.0,2.9,0.9,1.0,7.0,3.9,8.0,3.1,3.0]

shipping_cost = [100,150,0,60,75,50,120,110,90,130,115]
quantity = [1008,600,1800,252,126,315,882,252,450,315,270]
price = [550,341,825,660,660,688,484,660,330,506,556]

model = ConcreteModel()
model.dual = Suffix(direction = Suffix.IMPORT)
model.x = Var(range(11),range(3),within= NonNegativeReals)

# Define Objective Function
model.obj = Objective(
    expr = sum((price[i] + shipping_cost[i])*model.x[i,j] for i in range(11) for j in range(3)), 
    sense = minimize)

# capacity
def capacity_rule(model,i):
    return sum(model.x[i,j] for j in range(3)) <= quantity[i]

model.capacity_con = Constraint(range(11), rule=capacity_rule)

# demand
def demand_rule(model,j):
    return sum(model.x[i,j] for i in range(11)) == demand[j]

model.demand_con = Constraint(range(3), rule=demand_rule)

# color1 -- -1
def color1_rule(model,j):
    return (sum(color[i]*model.x[i,j]/demand[j] for i in range(11)) >= 3.5)
model.color1_con = Constraint(range(3), rule=color1_rule)

# color2
def color2_rule(model,j):
    return (sum(color[i]*model.x[i,j]/demand[j] for i in range(11)) <= 5.6)
model.color2_con = Constraint(range(3), rule=color2_rule)

# acid1
def acid1_rule(model,j):
    return (sum(acid[i]*model.x[i,j]/demand[j] for i in range(11)) >= 0.75)
model.acid1_con = Constraint(range(3), rule=acid1_rule)

# acid2 -- +1
def acid2_rule(model,j):
    return (sum(acid[i]*model.x[i,j]/demand[j] for i in range(11)) <= 2)
model.acid2_con = Constraint(range(3), rule=acid2_rule)

# brix1
def brix1_rule(model,j):
    return (sum(brix[i]*model.x[i,j]/demand[j] for i in range(11)) >= 11.5)
model.brix1_con = Constraint(range(3), rule=brix1_rule)

# brix2
def brix2_rule(model,j):
    return (sum(brix[i]*model.x[i,j]/demand[j] for i in range(11)) <= 12.5)
model.brix2_con = Constraint(range(3), rule=brix2_rule)

# astringency
def astringency_rule(model,j):
    return (sum(astringency[i]*model.x[i,j]/demand[j] for i in range(11)) <= 4)
model.astringency_con = Constraint(range(3), rule=astringency_rule)

# florida
def florida_rule(model,j):
    return (model.x[2,j] >= 0.35*demand[j])

model.florida_con = Constraint(range(3), rule=florida_rule)

# x greater than 0
def x_rule(model,i,j):
    return model.x[i,j] >= 0

model.x_con = Constraint(range(11),range(3), rule=x_rule)

# solve model
opt = SolverFactory('glpk').solve(model)

res = pd.DataFrame()
for i in range(11):
    for j in range(3):
        res.at[i,j] = value(model.x[i,j])
        

res = res.set_axis(["Brazil","India","Florida","California","Arizona","Texas",
                    "China","Spain","Mexico","Egypt","Italy"], axis=0)
res = res.set_axis(["Jan", "Feb", "Mar"],axis = 1)

print('Total Cost:', value(model.obj))

print('===========================================')
print(res)
print('-------------------------------------------')

print('Dual Solutions')
print("dual solution of astringency", mean(model.dual[model.astringency_con[i]] for i in range(3)))
print("dual solution of brix upper limit", mean(model.dual[model.brix2_con[i]] for i in range(3)))
print("dual solution of brix lower limit", mean(model.dual[model.brix1_con[i]] for i in range(3)))
print("dual solution of acid upper limit", mean(model.dual[model.acid2_con[i]] for i in range(3)))
print("dual solution of acid lower limit", mean(model.dual[model.acid1_con[i]] for i in range(3)))
print("dual solution of color upper limit", mean(model.dual[model.color2_con[i]] for i in range(3)))
print("dual solution of color lower limit", mean(model.dual[model.color1_con[i]] for i in range(3)))

Total Cost: 1938966.7817371932
                   Jan         Feb         Mar
Brazil      110.771185   63.760598  296.935922
India         0.000000   98.134744    0.000000
Florida     315.000000  315.000000  367.500000
California    0.000000    0.000000    0.000000
Arizona       0.000000    0.000000    0.000000
Texas         0.000000    0.000000    0.000000
China         0.000000    0.000000    0.000000
Spain        41.037356    0.000000  206.860194
Mexico      169.555990  101.740126  178.703883
Egypt       263.635468   51.364532    0.000000
Italy         0.000000  270.000000    0.000000
-------------------------------------------
Dual Solutions
dual solution of astringency -9965.478841870881
dual solution of brix upper limit 0.0
dual solution of brix lower limit 27547.884187082334
dual solution of acid upper limit 0.0
dual solution of acid lower limit 0.0
dual solution of color upper limit 0.0
dual solution of color lower limit 0.0


In [45]:
from pyomo.environ import *
import pandas as pd

demand = [900, 900, 1050] # predicted demand
color = [3.0,1.0,2.9,4.9,5.0,5.3,3.0,7.9,3.0,5.0,9.0]
acid = [0.60,1.42,0.91,1.02,0.70,0.70,1.38,1.12,1.31,1.30,0.50] # in percentage
brix = [10.6,6.6,12.0,11.0,11.8,10.1,9.0,15.1,7.8,13.1,14.0]
astringency = [2.9,7.0,3.0,2.9,0.9,1.0,7.0,3.9,8.0,3.1,3.0]

shipping_cost = [100,150,0,60,75,50,120,110,90,130,115]
quantity = [1008,600,1800,252,126,315,882,252,450,315,270]
price = [550,341,825,660,660,688,484,660,330,506,556]

model = ConcreteModel()
model.dual = Suffix(direction = Suffix.IMPORT)
model.x = Var(range(11),range(3),within= NonNegativeReals)

# Define Objective Function
model.obj = Objective(
    expr = sum((price[i] + shipping_cost[i])*model.x[i,j] for i in range(11) for j in range(3)), 
    sense = minimize)

# capacity
def capacity_rule(model,i):
    return sum(model.x[i,j] for j in range(3)) <= quantity[i]

model.capacity_con = Constraint(range(11), rule=capacity_rule)

# demand
def demand_rule(model,j):
    return sum(model.x[i,j] for i in range(11)) == demand[j]

model.demand_con = Constraint(range(3), rule=demand_rule)

# color1 -- -1.2(2)
def color1_rule(model,j):
    return (sum(color[i]*model.x[i,j]/demand[j] for i in range(11)) >= 3.3)
model.color1_con = Constraint(range(3), rule=color1_rule)

# color2
def color2_rule(model,j):
    return (sum(color[i]*model.x[i,j]/demand[j] for i in range(11)) <= 5.6)
model.color2_con = Constraint(range(3), rule=color2_rule)

# acid1
def acid1_rule(model,j):
    return (sum(acid[i]*model.x[i,j]/demand[j] for i in range(11)) >= 0.75)
model.acid1_con = Constraint(range(3), rule=acid1_rule)

# acid2 -- +1(1)
def acid2_rule(model,j):
    return (sum(acid[i]*model.x[i,j]/demand[j] for i in range(11)) <= 2)
model.acid2_con = Constraint(range(3), rule=acid2_rule)

# brix1 -- -1(2)
def brix1_rule(model,j):
    return (sum(brix[i]*model.x[i,j]/demand[j] for i in range(11)) >= 10.5)
model.brix1_con = Constraint(range(3), rule=brix1_rule)

# brix2
def brix2_rule(model,j):
    return (sum(brix[i]*model.x[i,j]/demand[j] for i in range(11)) <= 12.5)
model.brix2_con = Constraint(range(3), rule=brix2_rule)

# astringency -- +0.1(2)
def astringency_rule(model,j):
    return (sum(astringency[i]*model.x[i,j]/demand[j] for i in range(11)) <= 4.1)
model.astringency_con = Constraint(range(3), rule=astringency_rule)

# florida
def florida_rule(model,j):
    return (model.x[2,j] >= 0.35*demand[j])

model.florida_con = Constraint(range(3), rule=florida_rule)

# x greater than 0
def x_rule(model,i,j):
    return model.x[i,j] >= 0

model.x_con = Constraint(range(11),range(3), rule=x_rule)

# solve model
opt = SolverFactory('glpk').solve(model)

res = pd.DataFrame()
for i in range(11):
    for j in range(3):
        res.at[i,j] = value(model.x[i,j])
        

res = res.set_axis(["Brazil","India","Florida","California","Arizona","Texas",
                    "China","Spain","Mexico","Egypt","Italy"], axis=0)
res = res.set_axis(["Jan", "Feb", "Mar"],axis = 1)

print('Total Cost:', value(model.obj))

print('===========================================')
print(res)
print('-------------------------------------------')

print('Dual Solutions')
print("dual solution of astringency", mean(model.dual[model.astringency_con[i]] for i in range(3)))
print("dual solution of brix upper limit", mean(model.dual[model.brix2_con[i]] for i in range(3)))
print("dual solution of brix lower limit", mean(model.dual[model.brix1_con[i]] for i in range(3)))
print("dual solution of acid upper limit", mean(model.dual[model.acid2_con[i]] for i in range(3)))
print("dual solution of acid lower limit", mean(model.dual[model.acid1_con[i]] for i in range(3)))
print("dual solution of color upper limit", mean(model.dual[model.color2_con[i]] for i in range(3)))
print("dual solution of color lower limit", mean(model.dual[model.color1_con[i]] for i in range(3)))

Total Cost: 1885102.1975806449
                   Jan         Feb         Mar
Brazil      330.147059  234.573529  159.997154
India         0.000000    0.000000  231.492944
Florida     315.000000  315.000000  367.500000
California    0.000000    0.000000    0.000000
Arizona       0.000000    0.000000    0.000000
Texas         0.000000    0.000000    0.000000
China         0.000000    0.000000    0.000000
Spain         0.000000    0.000000    0.000000
Mexico      204.602941  199.676471   45.720588
Egypt         0.000000  150.750000  164.250000
Italy        50.250000    0.000000   81.039315
-------------------------------------------
Dual Solutions
dual solution of astringency -34935.4838709677
dual solution of brix upper limit 0.0
dual solution of brix lower limit 0.0
dual solution of acid upper limit 0.0
dual solution of acid lower limit 0.0
dual solution of color upper limit 0.0
dual solution of color lower limit 3907.25806451612


# Q1-4

In [4]:
from pyomo.environ import *
import pandas as pd

demand = [900, 900, 1050] # predicted demand
color = [3.0,1.0,2.9,4.9,5.0,5.3,3.0,7.9,3.0,5.0,9.0]
acid = [0.60,1.42,0.91,1.02,0.70,0.70,1.38,1.12,1.31,1.30,0.50] # in percentage
brix = [10.6,6.6,12.0,11.0,11.8,10.1,9.0,15.1,7.8,13.1,14.0]
astringency = [2.9,7.0,3.0,2.9,0.9,1.0,7.0,3.9,8.0,3.1,3.0]

shipping_cost = [100,150,0,60,75,50,120,110,90,130,115]
quantity = [1008,600,1800,252,126,315,882,252,450,315,270]
price = [550,341,825,660,660,688,484,660,330,506,556]

model = ConcreteModel()

model.x = Var(range(11),range(3),within= NonNegativeReals)
model.y = Var(range(11),range(3),within= Binary)

# Define Objective Function
model.obj = Objective(
    expr = sum((price[i] + shipping_cost[i])*model.x[i,j] 
               for i in range(11) for j in range(3)), 
    sense = minimize)

# capacity
def capacity_rule(model,i):
    return sum(model.x[i,j] for j in range(3)) <= quantity[i]

model.capacity_con = Constraint(range(11), rule=capacity_rule)

# demand
def demand_rule(model,j):
    return sum(model.x[i,j] for i in range(11)) == demand[j]

model.demand_con = Constraint(range(3), rule=demand_rule)

# color1
def color1_rule(model,j):
    return (sum(color[i]*model.x[i,j]/demand[j] for i in range(11)) >= 4.5)
    
model.color1_con = Constraint(range(3), rule=color1_rule)

# color2
def color2_rule(model,j):
    return (sum(color[i]*model.x[i,j]/demand[j] for i in range(11)) <= 5.6)

model.color2_con = Constraint(range(3), rule=color2_rule)

# acid1
def acid1_rule(model,j):
    return (sum(acid[i]*model.x[i,j]/demand[j] for i in range(11)) >= 0.75)

model.acid1_con = Constraint(range(3), rule=acid1_rule)

# acid2
def acid2_rule(model,j):
    return (sum(acid[i]*model.x[i,j]/demand[j] for i in range(11)) <= 1)

model.acid2_con = Constraint(range(3), rule=acid2_rule)

# brix1
def brix1_rule(model,j):
    return (sum(brix[i]*model.x[i,j]/demand[j] for i in range(11)) >= 11.5)

model.brix1_con = Constraint(range(3), rule=brix1_rule)

# brix2
def brix2_rule(model,j):
    return (sum(brix[i]*model.x[i,j]/demand[j] for i in range(11)) <= 12.5)

model.brix2_con = Constraint(range(3), rule=brix2_rule)

# astringency
def astringency_rule(model,j):
    return (sum(astringency[i]*model.x[i,j]/demand[j] for i in range(11)) <= 4)

model.astringency_con = Constraint(range(3), rule=astringency_rule)

# florida
def florida_rule(model,j):
    return (model.x[2,j] >= 0.35*demand[j])

model.florida_con = Constraint(range(3), rule=florida_rule)

# x greater than 0
def x_rule(model,i,j):
    return model.x[i,j] >= 0
            
model.x_con = Constraint(range(11),range(3), rule=x_rule)

# no more than 4
def y_rule(model,j):
    return sum(model.y[i,j] for i in range(11)) <=4

model.y_con = Constraint(range(3), rule=y_rule)

# x and y
def x_y_rule(model,i,j):
    return model.y[i,j]>=model.x[i,j]/demand[j]

model.x_y_con = Constraint(range(11), range(3), rule=x_y_rule)

# solve model
opt = SolverFactory('glpk').solve(model)

res = pd.DataFrame()
for i in range(11):
    for j in range(3):
        res.at[i,j] = value(model.x[i,j])
res = res.set_axis(["Brazil","India","Florida","California","Arizona","Texas",
                    "China","Spain","Mexico","Egypt","Italy"], axis=0)
res = res.set_axis(["Jan", "Feb", "Mar"],axis = 1)        
       
print('Total Cost:', value(model.obj))

print('===========================================')
print(res)
print('-------------------------------------------')

Total Cost: 2033113.1655425637
                   Jan         Feb         Mar
Brazil        0.000000    0.000000    0.000000
India         0.000000    0.000000    0.000000
Florida     393.044152  346.360484  370.991474
California    0.000000  252.000000    0.000000
Arizona     126.000000    0.000000    0.000000
Texas         0.000000    0.000000  211.841657
China         0.000000    0.000000    0.000000
Spain         0.000000    0.000000  252.000000
Mexico      191.405112    0.000000  215.166870
Egypt         0.000000  221.190252    0.000000
Italy       189.550736   80.449264    0.000000
-------------------------------------------


# Q2

In [1]:
from pyomo.environ import *


model = ConcreteModel()

model.dual = Suffix(direction = Suffix.IMPORT)
#define non-negative decision variables
model.x = Var(within=NonNegativeReals)
model.y = Var(within=NonNegativeReals)

#define objective function
model.obj = Objective(
    expr = 3*model.x + 5*model.y, 
    sense = maximize )

# x
model.model_x = Constraint(
    expr = model.x <=8)

# y
model.model_y = Constraint(
    expr = model.y <=12)

#Contractual Requirement
model.x_y = Constraint(
    expr =  3*model.x + 2*model.y <=36)

#Contractual Requirement
model.CRDN_con = Constraint(
    expr = model.x >=0)

model.CRDT_con = Constraint(
    expr = model.y >=0)

#Solve the model
opt = SolverFactory('glpk').solve(model)

# Print the solutions
print('Final Results:', value(model.obj))

print('x: ' ,   value(model.x))
print('y: ' ,   value(model.y))
print("Dual Solution")

print("dual solution of x", model.dual[model.model_x])
print("dual solution of y", model.dual[model.model_y])


Final Results: 72.0
x:  4.0
y:  12.0
Dual Solution
dual solution of x 0.0
dual solution of y 3.0
